#### Chrysanthos Lianos
#### MSc Data Science 2016 Part Time
#### Machine Learning and Computational statistics Project

### First Part

In [1]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image as image
import pandas as pd
import scipy 
%matplotlib inline

In [2]:
def getdf(type):
    tables = []
    for i in range(10):
        temp_df = pd.read_csv('D:/mnistdata/{}{}.txt'.format(str(type),i),sep = ' ', names = range(784))
        temp_df['target'] = i
        tables.append(temp_df)
    tables = pd.concat(tables)
    return tables

Now get the data
train_X is the set of training data, test_X is test
train_Y is the OneHot column for train_X and also has column which is aggregation of values eg 0: [1.0.0.0.0.0.0.0.0]

In [3]:
enc = OneHotEncoder(dtype='int') 
# train
train_X = getdf('train')
train_Y = np.asmatrix(enc.fit_transform(train_X['target'].reshape(-1,1)).toarray())
# test
test_X = getdf('test')
test_Y =  np.asmatrix(enc.fit_transform(test_X['target'].reshape(-1,1)).toarray())
# train_Y = pd.DataFrame(enc.fit_transform(train_X['target'].reshape(-1,1)).toarray()) #training for Y
# train_Y['aggr'] = train_Y.apply(lambda row: ''.join(map(str, row)), axis=1)
# in order to access eg the first row of the train_X use .iloc[0]
# to see all rows whose target is in range (0..9) use train_X.loc[trainX['target'] == 0]
# train_X.loc[train_X['target']== 0].sample(5)
# len(train_X.loc[train_X['target']== 1])

Get train and validation as subset from train

In [4]:
# the total number of rows in training files and a random size of 10000 
whole = np.arange(train_X.shape[1]) 
random = np.random.randint(low=0, high=60000, size=10000)
# validation dataframe
validation = train_X.iloc[random,:] #works
# remove indices from whole training dataset that are in validation dataframe
train_random = np.delete(whole,[random])
train = train_X.iloc[train_random,:]

So we have VALIDATION, TRAIN, and then TEST

In [5]:
# validation
validation.sample(10)

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,target
4219,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
498,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
824,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
2546,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6
5955,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
989,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
4426,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1984,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
334,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1143,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5


In [6]:
# train
train.sample(10)

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,target
399,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
142,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
408,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
316,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
415,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
731,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
53,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
280,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
344,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
380,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


get our X for logistic regression, insert 1 in front of each row of flattened digit, and init params

In [7]:
# X is train_X.ix[:,:784] and Y is train_X['target']
X = train_X.ix[:,:784]
# add 1 infront of X
X.insert(0,'ones',1)
t = train_X['target']
N,D = X.shape
# set params
l = 0.5
winit=np.random.rand(785,1)
# winit = np.ones(D)
options = [500,1/1000000 ,8/N]  # accuracy of 6 decimal places

# attempt to normalize X to X2
X2 = X.apply(lambda x: x/255)

In [8]:
def sigmoid(z):
    yx = 1.0/(1+np.exp(-z))
    return yx

def ml_logregTrain(t,X,l,winit,options):
    w = winit
    iterations = options[0]
    tol = options[1]
    eta = options[2]
    ewold = float('-inf')
    for it in range(0,iterations):
        yx = np.dot(X,w.T)
        s = sigmoid(yx)
        ew = sum(np.matmul(t.T,s)) - np.log(sum(np.exp(s)))
        print('Iteration: {}, Cost function: {}'.format(it,ew))
        if np.absolute(ew -ewold) < tol:
            break
        gradient = np.dot((t-s).T,X) - np.multiply(l,w)
        print(gradient.shape,eta)
        #w += w += np.multiply(np.float(eta),gradient)
        w += np.multiply(np.float(eta),np.reshape(gradient,(1)))
        ewold=ew
    ew =0
    return w 

#w = ml_logregTrain(train_Y,X,l,winit,options)
#w = ml_logregTrain(train_Y,X2,l,np.reshape(winit,(len(winit),1)).T,options)
#w = ml_logregTrain(train_Y,X2,l,np.reshape(winit,(len(winit),1)).T,options)
#print(w)